# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачайте любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat.

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [2]:
#Reading the data file
checkins = pd.read_csv('checkins.dat', sep='|', skipinitialspace=True, skiprows=[1], low_memory=False)
print(checkins.info())
checkins.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021967 entries, 0 to 1021966
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1021967 non-null  object 
 1   user_id           1021966 non-null  float64
 2   venue_id          1021966 non-null  float64
 3   latitude          396634 non-null   float64
 4   longitude         396634 non-null   float64
 5   created_at        1021966 non-null  object 
dtypes: float64(4), object(2)
memory usage: 46.8+ MB
None


,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [3]:
#Stripping columns names (deleting whitespaces)
checkins.columns = checkins.columns.str.strip()

#Dropping rows with NaN values
checkins = checkins.dropna()
print('Shape without NaN: %s' % str(checkins.shape))
checkins.head()

Shape without NaN: (396634, 6)


,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift. Его описание при желании можно посмотреть в sklearn user guide. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [4]:
bandwidth = 0.1
subset = checkins.head(100000)

clustering = MeanShift(bandwidth=bandwidth)
clustering.fit(subset[['latitude', 'longitude']])
clustering.labels_

array([ 5,  7, 30, ..., 25, 19,  4])

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [6]:
subset['cluster_num'] = clustering.labels_
counts = subset['cluster_num'].value_counts()
fift = counts[counts > 15].index
subset = subset[subset['cluster_num'].isin(fift)]

C:\Users\Leoni\AppData\Local\Temp\ipykernel_28144\2274092738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['cluster_num'] = clustering.labels_


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [5]:
def distance(x, y):
    return np.sqrt( np.sum((x - y)**2) )

In [7]:
offices = np.array([
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134]
])

centers = clustering.cluster_centers_
distances = np.array([np.min([distance(center, office) for office in offices]) for center in centers])
near = centers[np.argsort(distances)]
minimal_distance = np.sort(distances)[0]

Выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [9]:
print('Minimal distance: %f' % minimal_distance)
print('Coordinates of closest centroid: %s' % near[:20][0])

Minimal distance: 0.007835
Coordinates of closest centroid: [-33.86063043 151.20477593]
